Example of PySpark ML

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Missing").getOrCreate()

In [6]:
#Read dataset
training = spark.read.csv("test1.csv", header=True, inferSchema=True)

In [7]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [8]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [10]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(
    inputCols=["age", "Experience"],
    outputCol="Independent Feature")

In [11]:
output = featureassembler.transform(training)

In [12]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent Feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [13]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Feature']

In [14]:
finalized_data = output.select("Independent Feature", "Salary")

In [15]:
finalized_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [16]:
from pyspark.ml.regression import LinearRegression
# train test split
train, test = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol="Independent Feature", labelCol="Salary")
regressor = regressor.fit(train)

In [17]:
# coefficients
regressor.coefficients

DenseVector([-263.7076, 1767.624])

In [19]:
# intercept
regressor.intercept

19919.060052212404

In [20]:
# prediction
pred = regressor.evaluate(test)

In [21]:
pred.predictions.show()

+-------------------+------+-----------------+
|Independent Feature|Salary|       prediction|
+-------------------+------+-----------------+
|         [29.0,4.0]| 20000|19342.03655352618|
+-------------------+------+-----------------+



In [22]:
pred.meanAbsoluteError, pred.meanSquaredError

(657.9634464738192, 432915.89689570636)